In [26]:
#import library untuk proses feature engineering
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams["figure.figsize"]= 20,10
%matplotlib inline

In [27]:
#membaca data hasil cleaning data dalam format csv
data5 = pd.read_csv("PriceHouseData_Cleaning.csv")
data5.head(10)

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2
5,Whitefield,2 BHK,1170.0,2.0,38.00,2
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3
9,Gandhi Bazar,6 Bedroom,1020.0,6.0,370.00,6


In [28]:
#pada pasar real estate harga price_per_sqrt sangat penting sehingga kita akan menentukan nilai tersebut dari perkalian total_sqft dan price
data5['price_per_sqrt'] = data5['price']*100000/data5['total_sqft']
data5.head()
#terlihat ada kolom price_per_sqrt tambahan pada dataset

,location,size,total_sqft,bath,price,bhk,price_per_sqrt
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


In [29]:
#pada tahap ini kita akan melakukan beberapa teknik feature engineering dan membuat fitur baru yang sangat meolong untuk mendeteksi outlier dan penghapusan pada tahap selanjutnya

In [30]:
#kita akan memulai feature engeneering dengan check fitur location. ada berapa location dan berapa baris pada tiap-tiap lokasi
#location adalah suatu fitur category yang tiap daerah memiliki info data pajak 
print('isi dari kolom location :  \n\n', data5.location.unique())
print()
print('jumlah dari feature yang terkandung dalam tiap type location \n\n : ', len(data5.location.unique()))
#terlihat kita memiliki 1200 data/fitur dan ini akan menjadi masalah ketika kita mencoba membuat dummy column menggunakan one-hot coding ditahap nantinya
#permasalahan ini kita kenal dengan istilah high-dimensionality problem
#ada teknik untuk mengurangi dimensi, salah satunya yang paling efektif adalah other category yang artinya ada fitur yang sama mengisi bayak baris
#dalam artian sederhana kita menggabungkan fitur sejenis yang terpisah

isi dari kolom location :  

 ['Electronic City Phase II' 'Chikka Tirupathi' 'Uttarahalli' ...
 '12th cross srinivas nagar banshankari 3rd stage' 'Havanur extension'
 'Abshot Layout']

jumlah dari feature yang terkandung dalam tiap type location 

 :  1298


In [31]:
#cara melihat fitur tersebut yang pada tahap ini kita telah susun dari tertinggi hingga terendah
data5.location = data5.location.apply(lambda x: x.strip())
location_stats = data5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_stats


location
Whitefield              533
Sarjapur  Road          392
Electronic City         304
Kanakpura Road          264
Thanisandra             235
                       ... 
Kumbhena Agrahara         1
Kudlu Village,            1
Konappana Agrahara        1
Kodanda Reddy Layout      1
1 Annasandrapalya         1
Name: location, Length: 1287, dtype: int64

In [32]:
#pada tahapan ini kita ingin mengetahui location yang memiliki data kurang sama dengan 10
len(location_stats[location_stats<=10])
#info ini menyatakan bahwa ada 1047 location diluar 1287 yang mana mempunyai data kurang sama dengan 10

1047

In [33]:
# tahapan ini kita pisahkan data <= 10 
location_stats_less_than_10 = location_stats[location_stats<=10]
location_stats_less_than_10

location
Dodsworth Layout        10
BTM 1st Stage           10
Sadashiva Nagar         10
Thyagaraja Nagar        10
Kalkere                 10
                        ..
Kumbhena Agrahara        1
Kudlu Village,           1
Konappana Agrahara       1
Kodanda Reddy Layout     1
1 Annasandrapalya        1
Name: location, Length: 1047, dtype: int64

In [34]:
#disini kita mempunyai 1287 location sebagai unique location dan kita akan menerapkan tranformasi ke dalam dataframe ini
len(data5.location.unique())


1287

In [35]:
#pada tahapan ini dapat kita katakan mengelopokkan atau menfilter 
#kita dapat katakan bahwa location ini adalah "other" location jika X ada dalam location stats
#berapapun nilai yang kita simpan setelah tranformasi ini dilakukan, apa yang akan terjadi semua location ini akan dikonversi ke other location (lokasi lainya) dan kemudian ketika kita mencetak nilai unique location, kita akan melihat bahwa sekarang kita hanya memiliki 242 unique location 
#hal ini baik ketika nantinya kita akan lakukan one-hot coding karena kita memiliki lebih sedikit dimensi
data5.location = data5.location.apply(lambda x: 'other' if x in location_stats_less_than_10 else x)
data5.location.unique()
len(data5.location.unique())

241

In [36]:
#disini kita akan simpan dataset hasil proses feature engineeing (hal ini optional)
data5.to_csv('PriceHouseData_FeatureEngineering.csv', index=False)

In [37]:
data5 = pd.read_csv("PriceHouseData_FeatureEngineering.csv")
data5.head()


,location,size,total_sqft,bath,price,bhk,price_per_sqrt
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000
